In [2]:
from google.colab import drive
drive.mount('/content/drive')
training_file = '/content/drive/MyDrive/all/ML/Part3/obeyd.sher'
raw_text = open(training_file, 'r').read()
raw_text = raw_text.lower()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
print(raw_text[:200])

اگر داری تو عقل و دانش و هوش
بیا بشنو حدیث گربه و موش
بخوانم از برایت داستانی
که در معنای آن حیران بمانی
ای خردمند عاقل و دانا
قصه موش و گربه برخوانا
قصه موش و گربه مظلوم
گوش کن همچو در غلطانا
از قضای


In [5]:
all_words = raw_text.split()
unique_words = list(set(all_words))
print(f'Number of unique words: {len(unique_words)}')


Number of unique words: 7800


In [6]:
n_chars = len(raw_text)
print(f'Total characters: {n_chars}')


Total characters: 205991


In [7]:
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print(f'Total vocabulary (unique characters): {n_vocab}')
print(chars)


Total vocabulary (unique characters): 40
['\n', ' ', '-', 'ء', 'آ', 'ؤ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', '–']


In [8]:
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_index = dict((c, i) for i, c in enumerate(chars))
print(char_to_index)


{'\n': 0, ' ': 1, '-': 2, 'ء': 3, 'آ': 4, 'ؤ': 5, 'ا': 6, 'ب': 7, 'ة': 8, 'ت': 9, 'ث': 10, 'ج': 11, 'ح': 12, 'خ': 13, 'د': 14, 'ذ': 15, 'ر': 16, 'ز': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ل': 28, 'م': 29, 'ن': 30, 'ه': 31, 'و': 32, 'پ': 33, 'چ': 34, 'ژ': 35, 'ک': 36, 'گ': 37, 'ی': 38, '–': 39}


In [9]:
import numpy as np
seq_length = 160
n_seq = int(n_chars / seq_length)


In [10]:
X = np.zeros((n_seq, seq_length, n_vocab))
Y = np.zeros((n_seq, seq_length, n_vocab))


In [11]:
for i in range(n_seq):
  x_sequence = raw_text[i * seq_length : (i + 1) * seq_length]
  x_sequence_ohe = np.zeros((seq_length, n_vocab))
  for j in range(seq_length):
    char = x_sequence[j]
    index = char_to_index[char]
    x_sequence_ohe[j][index] = 1.
  X[i] = x_sequence_ohe
  y_sequence = raw_text[i * seq_length + 1 : (i + 1) * seq_length + 1] 
  y_sequence_ohe = np.zeros((seq_length, n_vocab))
  for j in range(seq_length):
    try:
      char = y_sequence[j]
    except:
      # just for ignoring out of range error
      print('final sentence')
    index = char_to_index[char]
    y_sequence_ohe[j][index] = 1.
  Y[i] = y_sequence_ohe


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses,optimizers
tf.random.set_seed(42)


In [13]:
hidden_units = 700
dropout = 0.4


In [14]:
batch_size = 100
n_epoch= 300

In [15]:
model = models.Sequential()
model.add(layers.LSTM(hidden_units, input_shape=(None, n_vocab), return_sequences=True, dropout=dropout))
model.add(layers.LSTM(hidden_units, return_sequences=True, dropout=dropout))
model.add(layers.TimeDistributed(layers.Dense(n_vocab, activation='softmax')))


In [16]:
optimizer = optimizers.RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [17]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 700)         2074800   
                                                                 
 lstm_1 (LSTM)               (None, None, 700)         3922800   
                                                                 
 time_distributed (TimeDistr  (None, None, 40)         28040     
 ibuted)                                                         
                                                                 
Total params: 6,025,640
Trainable params: 6,025,640
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping


In [19]:
file_path = "/content/drive/MyDrive/all/ML/Part3/weights/weights_epoch_{epoch:03d}_loss_{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')


In [20]:
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=50, verbose=1, mode='min')


In [21]:
def generate_text(model, gen_length, n_vocab, index_to_char):
  index = np.random.randint(n_vocab)
  y_char = [index_to_char[index]]
  X = np.zeros((1, gen_length, n_vocab))
  for i in range(gen_length):
    X[0, i, index] = 1.
    indices = np.argmax(model.predict( X[:, max(0, i - seq_length -1):i + 1, :])[0], 1)
    index = indices[-1]
    y_char.append(index_to_char[index])
  return ''.join(y_char)



In [22]:
class ResultChecker(Callback):
  def __init__(self, model, N, gen_length):
    self.model = model
    self.N = N
    self.gen_length = gen_length
    
  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.N == 0:
      result = generate_text(self.model, self.gen_length, n_vocab, index_to_char)
      print('\nMy War and Peace:\n' + result)


In [23]:
result_checker = ResultChecker(model, 10, 500)

In [24]:
# model.load_weights("/content/drive/MyDrive/all/ML/weights/weights_epoch_244_loss_1.6163.hdf5")
model.fit(X, Y, batch_size=batch_size, verbose=1, initial_epoch=0, epochs=n_epoch, callbacks=[result_checker, checkpoint, early_stop])


Epoch 1/300
13/13 [==============================] - ETA: 0s - loss: 3.6573
My War and Peace:
د                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

Epoch 1: loss improved from inf to 3.65727, saving model to /content/drive/MyDrive/all/ML/Part3/weights/weights_epoch_001_loss_3.6573.hdf5
13/13 [==============================] - 40s 3s/step - loss: 3.6573
Epoch 2/300
13/13 [==============================] - ETA: 0s - loss: 2.9764
Epoch 2: loss improved from 3.65727 to 2.97638, saving model to /content/drive/MyDrive/all/ML/Part3/weights/weights_epoc